Generiranje grafov:



In [3]:
import random
import pandas as pd
from sage.graphs.graph_generators import graphs
import numpy as np
import pandas as pd


#####################################################################

def SDCTD_stevilo2(g):
    h = g.complement()

    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    p.set_objective(sum([x[v] for v in g]))

    for v in g.vertices():
        neighbors = g.neighbors(v)  
        if neighbors:
            p.add_constraint(sum((x[w] + x[v]) for w in neighbors) >= 1)
        else:
            p.add_constraint(x[v] == 1)
            
    zaustavi = False

    for v in g.vertices():
        neighbors_complement = h.neighbors(v)  
        if neighbors_complement:  
            p.add_constraint(sum(x[w] for w in neighbors_complement) >= 1)

        else: 
            zaustavi = True
            break 

    if not zaustavi == True:
        p.solve()
        x = p.get_values(x)
        return(sum(1 for i in x.values() if i == 1))
    else:
        return None

#####################################################################


**Analiza, do nje pridemo s pomočjo spodnjih tabel:**


In [21]:
grafi = []  

for i in range(2, 9):
    for G in graphs.nauty_geng(f'{i} -c'):  
        grafi.append(G)

tabela = []

# Izračunamo minimalno in maksimalno stopnjo za vsak graf
for G in grafi:
    min_stopnja = min(G.degree())  
    max_stopnja = max(G.degree()) 

    dominacijsko_stevilo = SDCTD_stevilo2(G)
   
    n = G.order()  # Število vozlišč grafa   

##########################################################################

   
    E1 = dominacijsko_stevilo <= (1/2) * n

        
##########################################################################

    
    if min_stopnja == n - 1:
        E2 = dominacijsko_stevilo == None
    else:
        E2 = ''
        

##########################################################################
   
    if max_stopnja == n - 1:
        E3 = dominacijsko_stevilo == None
    else:
        E3 = ''
        
##########################################################################

    tabela.append([n, min_stopnja, max_stopnja, dominacijsko_stevilo, E1, E2, E3])


tabela_grafov = pd.DataFrame(tabela, columns=['Število vozlišč', 'Min stopnja', 'Max stopnja', 'Dominacijsko število', 'E1', 'E2', 'E3'])


false_counts = {}
true_counts = {}

# Iteriramo čez vse stolpce E1-E8
for i in range(1, 4):
    col = f'E{i}'
    false_counts[col] = (tabela_grafov[col] == False).sum()
    true_counts[col] = (tabela_grafov[col] == True).sum()

# Izpišemo rezultate
for i in range(1, 4):
    col = f'E{i}'
    false = false_counts[col]
    true = true_counts[col]
    total = true + false
    if total > 0:
        true_percent = (true / total) * 100
    else:
        true_percent = 0  # Za primer, ko je total enak 0
    print(f"Stolpec {col} - Število False: {false}, Število True: {true}, Delež False: {true_percent:.2f}%")

    
print(tabela_grafov)


Stolpec E1 - Število False: 128, Število True: 11984, Delež False: 98.94%
Stolpec E2 - Število False: 0, Število True: 7, Delež False: 100.00%
Stolpec E3 - Število False: 0, Število True: 1252, Delež False: 100.00%


       Število vozlišč  Min stopnja  Max stopnja Dominacijsko število     E1  \
0                    2            1            1                 None   True   
1                    3            1            2                 None   True   
2                    3            2            2                 None   True   
3                    4            1            3                 None   True   
4                    4            1            2                    2   True   
5                    4            1            3                 None   True   
6                    4            2            2                    4  False   
7                    4            2            3                 None   True   
8                    4            3            3                 None   True   
9                    5            1            4                 None   True   
10                   5            1            3                    3  False   
11                   5            1     

**Uredimo po številu vozlišč \(n\):**


In [19]:
grafi_po_n = {}  # Slovar, kjer bo ključ število vozlišč, vrednost pa seznam grafov

for i in range(3, 8):
    grafi_po_n[i] = []  # Inicializiramo seznam za grafe z n vozlišči
    for G in graphs.nauty_geng(f'{i} -c'):  # 'c' pomeni povezane grafe
        grafi_po_n[i].append(G)

tabele_po_n = {}  # Slovar za shranjevanje tabel za posamezne n

# Obdelamo grafe za vsak n in ustvarimo tabele
for n, grafi in grafi_po_n.items():
    tabela = []

    for G in grafi:
        min_stopnja = min(G.degree())  
        max_stopnja = max(G.degree()) 

        SDCTD_stevilo = SDCTD_stevilo2(G)

        tabela.append([n, min_stopnja, max_stopnja, SDCTD_stevilo])

    # Ustvarimo DataFrame za trenutno število vozlišč
    tabela_df = pd.DataFrame(tabela, columns=['Število vozlišč', 'Min stopnja', 'Max stopnja', 'SDCTD_stevilo'])

    # Urejanje tabele
    tabela_df = tabela_df.sort_values(by=['Min stopnja', 'Max stopnja'], ascending=[True, False])

    tabele_po_n[n] = tabela_df

# Izpis vseh tabel za posamezne n
for n, tabela in tabele_po_n.items():
    print(f"Tabela za grafe s {n} vozlišči (urejena):")
    print(tabela)
    print()



Tabela za grafe s 3 vozlišči (urejena):
   Število vozlišč  Min stopnja  Max stopnja SDCTD_stevilo
0                3            1            2          None
1                3            2            2          None

Tabela za grafe s 4 vozlišči (urejena):
   Število vozlišč  Min stopnja  Max stopnja SDCTD_stevilo
0                4            1            3          None
2                4            1            3          None
1                4            1            2             2
4                4            2            3          None
3                4            2            2             4
5                4            3            3          None

Tabela za grafe s 5 vozlišči (urejena):
    Število vozlišč  Min stopnja  Max stopnja SDCTD_stevilo
0                 5            1            4          None
2                 5            1            4          None
5                 5            1            4          None
15                5            1            4   

**Razvrsimo glede na minimalno stopnjo:**


In [18]:

grafi = []

# Generiranje grafov
for i in range(3, 8):
    for G in graphs.nauty_geng(f'{i} -c'): 
        grafi.append(G)

# Slovar za shranjevanje tabel za vsako minimalno stopnjo
tabele_po_min_stopnji = {}

# Izračun za vsak graf
for G in grafi:
    min_stopnja = min(G.degree())  
    max_stopnja = max(G.degree()) 
    SDCTD_stevilo = SDCTD_stevilo2(G)
    n = G.order()  # Število vozlišč grafa   


    # Če tabela za določeno min stopnjo še ne obstaja, jo ustvarimo
    if min_stopnja not in tabele_po_min_stopnji:
        tabele_po_min_stopnji[min_stopnja] = []

    # Dodajanje podatkov v ustrezno tabelo
    tabele_po_min_stopnji[min_stopnja].append([n, min_stopnja, max_stopnja, SDCTD_stevilo])

# Nastavimo, da se prikažejo vse vrstice
pd.set_option('display.max_rows', None)

# Shranjevanje tabel kot DataFrame in urejanje
for min_stopnja, tabela in tabele_po_min_stopnji.items():
    df = pd.DataFrame(tabela, columns=['Število vozlišč', 'Min stopnja', 'Max stopnja', 'SDCTD_stevilo'])
    
    # Urejanje najprej po številu vozlišč, nato po največji stopnji
    df.sort_values(by=['Število vozlišč', 'Max stopnja'], inplace=True)
    
    # Shranjevanje urejene tabele
    tabele_po_min_stopnji[min_stopnja] = df

# Izpis vseh tabel
for min_stopnja, df in tabele_po_min_stopnji.items():
    print(f"Tabela za minimalno stopnjo {min_stopnja}:\n")
    print(df)
    print("\n")

Tabela za minimalno stopnjo 1:

     Število vozlišč  Min stopnja  Max stopnja SDCTD_stevilo
0                  3            1            2          None
2                  4            1            2             2
1                  4            1            3          None
3                  4            1            3          None
10                 5            1            2             2
5                  5            1            3             3
7                  5            1            3             2
8                  5            1            3             3
11                 5            1            3             2
12                 5            1            3             2
4                  5            1            4          None
6                  5            1            4          None
9                  5            1            4          None
13                 5            1            4          None
35                 6            1            2       

**Razvrstimo glede maksimalno stopnjo:**


In [17]:

grafi = []

# Generiranje grafov
for i in range(3, 8):
    for G in graphs.nauty_geng(f'{i} -c'):  
        grafi.append(G)

# Slovar za shranjevanje tabel za vsako maksimalno stopnjo
tabele_po_max_stopnji = {}

# Izračun za vsak graf
for G in grafi:
    min_stopnja = min(G.degree())  
    max_stopnja = max(G.degree()) 
    SDCTD_stevilo = SDCTD_stevilo2(G)
    n = G.order()  # Število vozlišč grafa   
    

    # Če tabela za določeno max stopnjo še ne obstaja, jo ustvarimo
    if max_stopnja not in tabele_po_max_stopnji:
        tabele_po_max_stopnji[max_stopnja] = []

    # Dodajanje podatkov v ustrezno tabelo
    tabele_po_max_stopnji[max_stopnja].append([n, min_stopnja, max_stopnja, SDCTD_stevilo])

# Nastavimo, da se prikažejo vse vrstice
pd.set_option('display.max_rows', None)

# Shranjevanje tabel kot DataFrame in urejanje
for max_stopnja, tabela in tabele_po_max_stopnji.items():
    df = pd.DataFrame(tabela, columns=['Število vozlišč', 'Min stopnja', 'Max stopnja', 'SDCTD_stevilo'])
    
    # Urejanje najprej po številu vozlišč, nato po največji stopnji
    df.sort_values(by=['Število vozlišč', 'Max stopnja'], inplace=True)
    
    # Shranjevanje urejene tabele
    tabele_po_max_stopnji[max_stopnja] = df

# Izpis vseh tabel
for max_stopnja, df in tabele_po_max_stopnji.items():
    print(f"Tabela za maksimalno stopnjo {max_stopnja}:\n")
    print(df)
    print("\n")


Tabela za maksimalno stopnjo 2:

   Število vozlišč  Min stopnja  Max stopnja SDCTD_stevilo
0                3            1            2          None
1                3            2            2          None
2                4            1            2             2
3                4            2            2             4
4                5            1            2             2
5                5            2            2             3
6                6            1            2             2
7                6            2            2             2
8                7            1            2             3
9                7            2            2             3


Tabela za maksimalno stopnjo 3:

     Število vozlišč  Min stopnja  Max stopnja SDCTD_stevilo
0                  4            1            3          None
1                  4            1            3          None
2                  4            2            3          None
3                  4            3      